In [1]:
%update_token

Token successfully refreshed.


In [1]:
!env| grep -i mlflow

JPY_SESSION_NAME=/home/geun-tak.roh-hpe.com/geun-tak-roh-hp-117928e5/license_detection/finetune/3. Finetune with MLFlow.ipynb
MLFLOW_S3_ENDPOINT_URL=http://local-s3-service.ezdata-system.svc.cluster.local:30000
MLFLOW_S3_IGNORE_TLS=true
MLFLOW_TRACKING_INSECURE_TLS=true
MLFLOW_TRACKING_TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4dWVCbXhzdldzNkZUZ1B6dThmNG9HbTRldU1EYi13YUExZEhPT0g2SzZFIn0.eyJleHAiOjE3NDgyNDIzMzUsImlhdCI6MTc0ODI0MDUzNSwiYXV0aF90aW1lIjoxNzQ4MjIyNjgzLCJqdGkiOiIzZGVjODg3MS1iYTVhLTRiOTMtOTIwMi02YjM3MjgzZDI0ZGQiLCJpc3MiOiJodHRwczovL2tleWNsb2FrLmluZ3Jlc3MucGNhaTAzMDUuc2cyLmhwZWNvbG8ubmV0L3JlYWxtcy9VQSIsInN1YiI6ImZhZDM4N2U2LWJlYTEtNGRkYy1iM2ZlLTM4NzNlMjBhNTM1OSIsInR5cCI6IkJlYXJlciIsImF6cCI6InVhIiwibm9uY2UiOiJJby1qa2hJZUVOSG9Xa3dxNlk4dE1LTm5oNk9OMkwxYjdWOVRySzBUMlBrIiwic2Vzc2lvbl9zdGF0ZSI6ImQ4N2Q3MDY1LTE5MWYtNGZjZS1iOTRkLTZiMDM3NDZjYjU5NSIsImFjciI6IjEiLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIHNnMnMxNzNyMzA6dWEgZW1haWwgb2ZmbGluZV9hY2Nlc3MiLCJzaWQiOiJkODdkNzA2NS0xOTFmLTRmY2U

In [2]:
from ultralytics import YOLO
import os

In [3]:
model = YOLO("yolo11s")

In [4]:
datasets = "./datasets/data.yaml"

In [5]:
model.reset_callbacks()
def update_token(trainer):
    with open('/etc/secrets/ezua/.auth_token','r') as file:
        AUTH_TOKEN = file.read()
        os.environ['MLFLOW_TRACKING_TOKEN']=AUTH_TOKEN
        print("Token successfully refreshed.")

model.add_callback("on_model_save",update_token)
# dir(model.callbacks.items)
model.callbacks["on_model_save"]

[<function ultralytics.utils.callbacks.base.on_model_save(trainer)>,
 <function __main__.update_token(trainer)>]

In [6]:
run_name = model.model_name.name.replace('.pt','-') + time.strftime("%Y%m%d-%H%M%S", time.localtime())

os.environ['MLFLOW_RUN'] = run_name
os.environ['MLFLOW_TRACKING_URI'] = "https://mlflow.ingress.pcai0305.sg2.hpecolo.net"
os.environ['MLFLOW_EXPERIMENT_NAME'] = 'license_plate_yolo11s'

In [7]:
import mlflow
print("MLflow Tracking URI:", mlflow.get_tracking_uri())

MLflow Tracking URI: http://mlflow.mlflow.svc.cluster.local:5000


In [8]:
results = model.train(data=datasets, epochs=3)  # train the model

New https://pypi.org/project/ultralytics/8.3.145 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA L40S, 45589MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./datasets/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train10, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning /mnt/user/license_detection/finetune/datasets/train/labels.cache... 21173 images, 28 backgrounds, 0 corrupt: 100%|██████████| 21173/21173 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 11.2±3.1 MB/s, size: 35.9 KB)


val: Scanning /mnt/user/license_detection/finetune/datasets/valid/labels.cache... 2046 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2046/2046 [00:00<?, ?it/s]


Plotting labels to runs/detect/train10/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)


...

AUTH TOKEN update is done!

3 epochs completed in 0.076 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 19.2MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics 8.3.144 🚀 Python-3.11.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA L40S, 45589MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:05<00:00, 12.40it/s]


                   all       2046       2132      0.955      0.922      0.969      0.642
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train10


2025/05/26 06:02:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run yolo11s-20250526-055714 at: https://mlflow.ingress.pcai0305.sg2.hpecolo.net/#/experiments/1/runs/b087c4f05f684bcaa1d823e3e70185bf.
2025/05/26 06:02:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow.ingress.pcai0305.sg2.hpecolo.net/#/experiments/1.


MLflow: results logged to https://mlflow.ingress.pcai0305.sg2.hpecolo.net
MLflow: disable with 'yolo settings mlflow=False'


In [12]:
results.save_dir.joinpath("results.csv")

PosixPath('runs/detect/train10/results.csv')

In [13]:
from ultralytics.utils.plotting import plot_results
plot_results(results.save_dir.joinpath("results.csv"))